This Python script reads in all the quality controlled csv files for each cast, computes GSW metrics and potential density, and saves to one new csv (final_qc_CTD_data_all.csv)

In [1]:
import pandas as pd
import numpy as np
import gsw
from glob import glob
# import re
import csv

In [2]:
source_dir = '../data_ctd_raw/'
save_dir = '../processed_data/'

In [3]:
fileList = glob(source_dir + 'SKQ202310S_ct1/*.csv')

print(fileList)

['../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s098c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s287c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s111c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s126c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s278c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s005c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s032c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s173c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s144c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s067c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s050c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s190c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s084c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s253c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s264c01_ct1.csv', '../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s019c01_ct1.csv', '../data_ctd_raw/SKQ202

In [4]:
def read_cast_metadata(filename_string):
    # make empty list to store cast metadata:
    data_strings = []
    # add the filename of the cast file to the cast metadata list
    data_strings.append(f'FILE = {filename_string}')

    # Open the CSV file for the cast
    with open(filename_string, 'r') as file:
    # Create a CSV reader object
        reader = csv.reader(file)
    
    # Read the cast metadata lines (3-10) and append to the data_strings list
        for i, row in enumerate(reader):
            if i >= 3 and i <= 10:
                data_strings.append(row[0].strip()) # .strip() removes whitespace
            elif 1 > 10:
                break
    
    # Split each data string into key and value
    split_data = [item.split(' = ') for item in data_strings]

    # Create DataFrame
    meta_df_2col = pd.DataFrame(split_data, columns=['Key', 'Value'])

    # # Strip Whitespace
    # meta_df_2col.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Transpose the DataFrame
    meta_df= meta_df_2col.set_index('Key').T

    return(meta_df)

In [5]:
def read_cast_data(filename_string):
    # The first part of this function generates column names for the dataframe, the second reads in the data
    # Variable Names are in the 11th row, Units are in the 12th, and data is row 13 on
    # We will read in varnames and units, combine the strings, then use these new strings as colnames for the data

    # variable names are in the 11th row
    varnames = pd.read_csv(filename_string, header=None, skiprows=11, nrows=1)

    # units for each variable are in the 12th row
    units = pd.read_csv(filename_string, header=None, skiprows=12, nrows=1)

    # create empty list to store combined varname and unit string
    column_names = []
    # combine the string
    for varnames, units in zip(varnames.values[0], units.values[0]):
    # if there is a unit, combine with varname to make new colname
        if pd.notna(units):
            column_names.append(str(varnames) + '_' + str(units))
    # if no unit, varname is colname
        else:
            column_names.append(str(varnames))

    # Here we read in the data
    # Read the CSV file skipping the first 13 rows, set the column names using list generated above
    data_df = pd.read_csv(filename_string, header=None, skiprows=13, names = column_names)

    return(data_df)

In [6]:
datadf = read_cast_data(fileList[2])
metadf = read_cast_metadata(fileList[2])

datadf.assign(**metadf.iloc[0])

,CTDPRS_DBAR,CTDPRS_FLAG_W,CTDDEPTH_METERS,CTDDEPTH_FLAG_W,CTDTMP_ITS-90,CTDTMP_FLAG_W,CTDTMP2_ITS-90,CTDTMP2_FLAG_W,CTDSAL_PSU,CTDSAL_FLAG_W,...,SPAR_FLAG_W,FILE,SECT_ID,STNNBR,CASTNO,DATE,TIME,LATITUDE,LONGITUDE,DEPTH
0,2.000,2.0,1.980,2.0,-0.5160,2.0,-0.5160,2.0,32.1543,2.0,...,5.0,../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s111...,HRE4-5.5,111,1,20230710,0253,70.8783,-163.275,45
1,3.000,2.0,2.969,2.0,-0.5321,2.0,-0.5396,2.0,32.1936,2.0,...,5.0,../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s111...,HRE4-5.5,111,1,20230710,0253,70.8783,-163.275,45
2,4.000,2.0,3.959,2.0,-0.7986,2.0,-0.8102,2.0,32.2448,2.0,...,5.0,../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s111...,HRE4-5.5,111,1,20230710,0253,70.8783,-163.275,45
3,5.000,2.0,4.949,2.0,-1.0970,2.0,-1.1035,2.0,32.2713,2.0,...,5.0,../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s111...,HRE4-5.5,111,1,20230710,0253,70.8783,-163.275,45
4,6.000,2.0,5.939,2.0,-1.2495,2.0,-1.2525,2.0,32.2899,2.0,...,5.0,../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s111...,HRE4-5.5,111,1,20230710,0253,70.8783,-163.275,45
5,7.000,2.0,6.929,2.0,-1.3086,2.0,-1.3100,2.0,32.3003,2.0,...,5.0,../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s111...,HRE4-5.5,111,1,20230710,0253,70.8783,-163.275,45
6,8.000,2.0,7.918,2.0,-1.3682,2.0,-1.3680,2.0,32.3143,2.0,...,5.0,../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s111...,HRE4-5.5,111,1,20230710,0253,70.8783,-163.275,45
7,9.000,2.0,8.908,2.0,-1.4654,2.0,-1.4659,2.0,32.3348,2.0,...,5.0,../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s111...,HRE4-5.5,111,1,20230710,0253,70.8783,-163.275,45
8,10.000,2.0,9.898,2.0,-1.4884,2.0,-1.4880,2.0,32.3519,2.0,...,5.0,../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s111...,HRE4-5.5,111,1,20230710,0253,70.8783,-163.275,45
9,11.000,2.0,10.888,2.0,-1.5353,2.0,-1.5353,2.0,32.3810,2.0,...,5.0,../data_ctd_raw/SKQ202310S_ct1/SKQ202310S_s111...,HRE4-5.5,111,1,20230710,0253,70.8783,-163.275,45


In [7]:
# create empty list to store dataframe from each cast
list_of_cast_dataframes = []
# for loop generates a dataframe for each cast, then appends to the list
for filename in fileList:
    # Read in Metadata using function
    metadata = read_cast_metadata(filename)
    # Read in Data using function
    data = read_cast_data(filename)
    # Data and Metadata shapes are incompatible, need to expand metadata so each dataframe has same number of rows
    # this will expand the metadata dataframe, duplicating the data to match the shape of the data dataframe
    metadata_expanded = pd.concat([metadata] * data.shape[0], ignore_index=True)
    # metadata_expanded = metadata.loc[data.index].reset_index(drop=True)
    # combine the data and metadata
    cast_dataframe = pd.concat([metadata_expanded, data], axis=1)
    # The concat caused some reshaping issues
    # Assign command allows us to assign the metadata variables to every row of the data dataframe
    # cast_dataframe = data.assign(**metadata.iloc[0])

    # One useless row at end of each file - must cut
    cast_dataframe.drop(cast_dataframe[cast_dataframe['CTDPRS_DBAR'] == 'END_DATA'].index, inplace = True)
    # append to list
    list_of_cast_dataframes.append(cast_dataframe)

In [8]:
# list_of_cast_dataframes[0]

In [9]:
# concatenate all the cast dataframes
ctd = pd.concat(list_of_cast_dataframes)

# Make STNNBR an int, CTDPRS_DBAR swap to numeric - both were sorting lexically
ctd['STNNBR'] = ctd['STNNBR'].astype(int)
ctd['CTDPRS_DBAR'] = pd.to_numeric(ctd['CTDPRS_DBAR'])

# after concat and drop, sort values by station then by pressure(depth), then reset indexing
ctd = ctd.sort_values(by = ['STNNBR', 'CTDPRS_DBAR'], ignore_index=True)

In [10]:
## Add SECTION and LINE variables using regex

# LINE is all letters before the first number in SECT_ID
ctd['LINE'] = ctd['SECT_ID'].str.extract(r'([a-zA-Z]+)(?=\d)', expand=False)

# SECTION is all characters before the dash, or, if there is no dash, all characters in SECT_ID
ctd['SECTION'] = ctd['SECT_ID'].str.split('-', expand=True)[0]

# check that all lines appear:
# ctd['LINE'].drop_duplicates()

# check that all sections appear:
# for i in ctd['SECTION'].drop_duplicates():
#     print(i)

In [11]:
# Convert Values to floats for use with GSW
ctd['CTDPRS_DBAR'] = ctd['CTDPRS_DBAR'].astype(float)
ctd['LONGITUDE'] = ctd['LONGITUDE'].astype(float)
ctd['LATITUDE'] = ctd['LATITUDE'].astype(float)
ctd['STNNBR'] = ctd['STNNBR'].astype(float)

In [12]:
# GSW (Gibbs SeaWater) Conversions Necessary to pull out Potential Density

ctd['SA_sal_gperkg'] = gsw.SA_from_SP(ctd['CTDSAL_PSU'], ctd['CTDPRS_DBAR'], ctd['LONGITUDE'], ctd['LATITUDE'])
ctd['CT_temp_its90'] = gsw.CT_from_t(ctd['SA_sal_gperkg'], ctd['CTDTMP_ITS-90'], ctd['CTDPRS_DBAR'])
ctd['SIGMA0_kgperm3'] = gsw.density.sigma0(ctd['SA_sal_gperkg'], ctd['CT_temp_its90'])
# ctd['rho_ref'] = 10
# ctd['poDensity_kgperm3'] = gsw.rho(ctd['SA_sal_gperkg'], ctd['CT_temp_its90'], ctd['rho_ref'])
# Coriolis parameter here = 2*OMEGA*np.sin(np.radians(latitude))
ctd['CORIOLIS'] = gsw.geostrophy.f(ctd['LATITUDE'])

In [13]:
# Write to CSV
ctd.to_csv(save_dir + 'final_qc_CTD_data.csv', index=False)